In [ ]:
import os
import json
import time
import openai
from underthesea import sent_tokenize
from langchain_text_splitters import CharacterTextSplitter
from tqdm import tqdm

# Thiết lập API key cho OpenAI từ biến môi trường
openai.api_key = os.getenv('OPENAI_API_KEY')

# Kiểm tra xem API key có được thiết lập hay không
if openai.api_key is None:
    raise ValueError("API key chưa được thiết lập. Vui lòng thiết lập biến môi trường 'OPENAI_API_KEY'.")

class VietnameseTextProcessor:
    def __init__(self, max_length: int = 2000, overlap: int = 200):
        self.max_length = max_length
        self.overlap = overlap
        
    def read_text_files(self, input_folder: str) -> list:
        """Đọc tất cả các file văn bản trong thư mục."""
        all_texts = []
        for filename in os.listdir(input_folder):
            if filename.endswith('.txt'):
                with open(os.path.join(input_folder, filename), 'r', encoding='utf-8') as f:
                    all_texts.append(f.read())
        return all_texts  # Trả về danh sách các văn bản

    def preprocess_text(self, text: str) -> str:
        """Tiền xử lý văn bản: giữ nguyên định dạng và dấu câu."""
        text = text.replace('\r\n', '\n')
        text = '\n'.join(line.strip() for line in text.split('\n'))
        return text
    
    def split_into_sentences(self, text: str) -> list:
        """Tách văn bản thành các câu sử dụng Underthesea."""
        sentences = sent_tokenize(text)
        valid_sentences = [s.strip() for s in sentences if len(s.split()) > 5]
        return valid_sentences
    
    def create_chunks(self, sentences: list) -> list:
        """Ghép các câu thành chunk sử dụng LangChain."""
        splitter = CharacterTextSplitter(
            chunk_size=self.max_length,
            chunk_overlap=self.overlap,
            separator="\n",
            length_function=len
        )
        text = '\n'.join(sentences)
        chunks = splitter.split_text(text)
        return chunks

    def generate_questions_and_answers(self, chunk: str) -> dict:
        """Gửi chunk tới GPT-4o-mini để tạo câu hỏi và câu trả lời."""
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "Bạn là một trợ lý hữu ích, am hiểu về Lịch sử Việt Nam giai đoạn 1975."},
                {"role": "user", "content": f"""
                    Đây là đoạn trích từ sách về Lịch sử Việt Nam giai đoạn 1975: {chunk} 
                    Từ đoạn trích, đóng vai người hỏi, hãy tạo 5 câu hỏi cho nội dung đoạn trích.
                    Đảm bảo rằng các câu hỏi có thể khám phá các khía cạnh khác nhau và đầy đủ nhất của nội dung. 
                    Từ các câu hỏi, đóng vai người trả lời, hãy tạo các câu trả tương ứng từ đoạn trích.
                    Đảm bảo chính xác thông tin của đoạn văn gốc nhất có thể.
                    Câu trả lời dài và đầy đủ nhất có thể, nhưng không dài hơn đoạn văn gốc.
                    Hãy trả về kết quả dưới dạng JSON như sau:
                    {{
                        "questions": [
                            "Câu hỏi 1",
                            "Câu hỏi 2"
                        ],
                        "answers": [
                            "Câu trả lời 1",
                            "Câu trả lời 2"
                        ]
                    }}
                """},
            ],
            temperature=0.5,
            response_format={"type": "json_object"}
        )

        questions_answers_json = response.choices[0].message.content.strip()

        # Kiểm tra xem nội dung có hợp lệ không
        if not questions_answers_json:
            raise ValueError("Nội dung phản hồi trống.")

        # Chuyển đổi chuỗi JSON thành danh sách
        try:
            data = json.loads(questions_answers_json)
            questions = data["questions"]
            answers = data["answers"]
            return {"questions": questions, "answers": answers}
        except json.JSONDecodeError as e:
            print("Lỗi phân tích cú pháp JSON:", e)
            raise ValueError("Nội dung phản hồi không phải là JSON hợp lệ.")

    def create_samples(self, questions: list, answers: list) -> list:
        """Tạo sample theo định dạng Alpaca."""
        samples = []
        for question, answer in zip(questions, answers):
            sample = {
                "instruction": "Hãy trả lời theo phong cách của thiền sư Thích Nhất Hạnh.",
                "input": question,
                "output": answer
            }
            samples.append(sample)
        return samples

def main():
    processor = VietnameseTextProcessor(max_length=2000)

    input_folder = "data-raw/Huy Duc"
    output_file = "output_samples.json"
    log_file = "error_log.log"  # Tên file log để ghi lại lỗi

    all_samples = []
    total_tokens = 0
    total_samples_processed = 0

    # Đọc tất cả văn bản từ các file trong thư mục
    all_texts = processor.read_text_files(input_folder)

    start_time = time.time()

    # Đơn giá cho input và output tokens
    input_price_per_million = 0.150 
    output_price_per_million = 0.600 

    # Khởi tạo tổng số token đầu vào và đầu ra
    total_request_tokens = 0
    total_response_tokens = 0

    # Xử lý từng file sách
    for idx, full_text in enumerate(all_texts):
        # Tiền xử lý văn bản
        processed_text = processor.preprocess_text(full_text)

        # Tách văn bản thành các câu
        sentences = processor.split_into_sentences(processed_text)

        # Tạo chunks từ các câu đã tách
        chunks = processor.create_chunks(sentences)

        # Xử lý từng chunk và tạo câu hỏi cùng câu trả lời
        for i, chunk in tqdm(enumerate(chunks), desc=f"Đang xử lý sách {idx + 1}", total=len(chunks)):
            try:
                qa_data = processor.generate_questions_and_answers(chunk)

                # Tính toán số token cho request và response
                request_tokens = len(chunk.split()) + sum(len(q.split()) for q in qa_data["questions"])
                response_tokens = sum(len(a.split()) for a in qa_data["answers"])

                total_request_tokens += request_tokens
                total_response_tokens += response_tokens

                total_tokens += (request_tokens + response_tokens)

                # Tạo sample cho từng câu hỏi và câu trả lời
                samples = processor.create_samples(qa_data["questions"], qa_data["answers"])
                
                # Lưu từng sample vào file JSON ngay sau khi tạo
                for sample in samples:
                    all_samples.append(sample)
                    with open(output_file, 'a', encoding='utf-8') as f:
                        json.dump(sample, f, ensure_ascii=False)
                        f.write('\n')  # Ghi thêm dòng mới sau mỗi sample

                total_samples_processed += len(samples)

            except Exception as e:
                with open(log_file, 'a', encoding='utf-8') as log_f:
                    log_f.write(f"Error processing chunk {i} of book {idx + 1}: {str(e)}\n")
                print(f"Đã xảy ra lỗi với chunk {i + 1} của sách {idx + 1}: {str(e)}")

    # Tính toán chi phí dựa trên số token đã sử dụng
    input_cost = (total_request_tokens / 1000000) * input_price_per_million
    output_cost = (total_response_tokens / 1000000) * output_price_per_million
    total_cost = input_cost + output_cost

    end_time = time.time()

    print(f"\nXử lý xong:")
    print(f"- Tổng số mẫu đã xử lý: {total_samples_processed}")
    print(f"- Tổng số token đã sử dụng: {total_tokens}")
    print(f"- Tổng số input tokens: {total_request_tokens}")
    print(f"- Tổng số output tokens: {total_response_tokens}")
    print(f"- Chi phí cho input tokens: ${input_cost:.3f}")
    print(f"- Chi phí cho output tokens: ${output_cost:.3f}")
    print(f"- Tổng chi phí: ${total_cost:.3f}")
    print(f"- Thời gian thực hiện: {end_time - start_time:.2f} giây")
    print(f"- Dataset được lưu tại: {output_file}")

if __name__ == "__main__":
3    main()

Đang xử lý sách 12: 100%|██████████| 407/407 [40:53<00:00,  6.03s/it]


Xử lý xong:
- Tổng số mẫu đã xử lý: 6632
- Tổng số token đã sử dụng: 1520771
- Tổng số input tokens: 1068841
- Tổng số output tokens: 451930
- Chi phí cho input tokens: $0.160
- Chi phí cho output tokens: $0.271
- Tổng chi phí: $0.431
- Thời gian thực hiện: 12623.06 giây
- Dataset được lưu tại: output_samples.json


In [8]:
import json

def fix_json_file(file_path):
    try:
        # Đọc nội dung từ file
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = f.readlines()

        # Xóa các dòng trống và tạo danh sách các đối tượng JSON
        json_objects = [json.loads(line) for line in lines if line.strip()]

        # Ghi lại vào file dưới dạng mảng JSON
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(json_objects, f, ensure_ascii=False, indent=2)

        print(f"File '{file_path}' đã được sửa thành công.")

    except Exception as e:
        print(f"Đã xảy ra lỗi khi sửa file: {e}")

# Gọi hàm với đường dẫn tới file cần sửa
fix_json_file('output_samples.json')

File 'output_samples.json' đã được sửa thành công.
